In [1]:
import zipfile
import os

# Define the path for the zip file and the extraction directory
zip_file_path = 'dataset.zip'
extract_dir = 'dataset'  # You can change this to your preferred extraction directory

# Create the extraction directory if it doesn't exist
os.makedirs(extract_dir, exist_ok=True)

# Extract the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Dataset extracted to {extract_dir}")


Dataset extracted to dataset


In [25]:
train_dir = os.path.join(extract_dir, 'train')
val_dir = os.path.join(extract_dir, 'valid')
test_dir = os.path.join(extract_dir, 'test')

print("Training Directory:", train_dir)
print("Validation Directory:", val_dir)
print("Test Directory:", test_dir)


Training Directory: dataset\train
Validation Directory: dataset\valid
Test Directory: dataset\test


In [26]:
import cv2
import os

# Function to normalize video resolution
def normalize_videos(video_paths, output_dir, width, height):
    os.makedirs(output_dir, exist_ok=True)  # Create output directory if it doesn't exist
    
    for video_path in video_paths:
        cap = cv2.VideoCapture(video_path)
        video_name = os.path.basename(video_path)  # Get the video name
        output_path = os.path.join(output_dir, f"normalized_{video_name}")  # Output path
        
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec
        out = cv2.VideoWriter(output_path, fourcc, 30.0, (width, height))
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            # Resize the frame
            frame = cv2.resize(frame, (width, height))
            out.write(frame)
        
        cap.release()
        out.release()
        print(f"Normalized video saved as: {output_path}")

# List of video files to normalize
video_files = ['video1.mp4', 'video2.mp4', 'video3.mp4']  # Replace with your video paths
output_directory = 'normalize'  # Specify output directory for normalized videos

# Normalize your videos
normalize_videos(video_files, output_directory, 640, 480)


Normalized video saved as: normalize\normalized_video1.mp4
Normalized video saved as: normalize\normalized_video2.mp4
Normalized video saved as: normalize\normalized_video3.mp4


In [27]:
pip install ultralytics

Note: you may need to restart the kernel to use updated packages.


In [7]:
from ultralytics import YOLO

In [8]:
model = YOLO('yolov8n.pt')  # Use a suitable pretrained model (e.g., 'yolov8s.pt' for small, 'yolov8m.pt' for medium)

100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:01<00:00, 3.31MB/s]


In [24]:
def process_video(video_path, output_path):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # Codec for output video
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (640, 480))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Perform inference
        results = model(frame)

        # Draw bounding boxes on the frame
        for result in results:
            boxes = result.boxes.xyxy  # Get the bounding box coordinates
            for box in boxes:
                x1, y1, x2, y2, conf, cls = box  # Unpack the box
                cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255, 0, 0), 2)  # Draw box
                cv2.putText(frame, f'{model.names[int(cls)]}: {conf:.2f}', (int(x1), int(y1) - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Write the processed frame to the output video
        out.write(frame)

    cap.release()
    out.release()

# Process each normalized video
for video in ['normalized_video1.mp4', 'normalized_video2.mp4', 'normalized_video3.mp4']:
    process_video(video, f'output_{video}')

In [10]:
import cv2
import numpy as np

def load_advertisement(image_path):
    new_ad = cv2.imread(image_path)
    return new_ad

def adjust_lighting(original_frame, new_ad):
    # Simple adjustment: histogram equalization
    original_yuv = cv2.cvtColor(original_frame, cv2.COLOR_BGR2YUV)
    new_ad_yuv = cv2.cvtColor(new_ad, cv2.COLOR_BGR2YUV)

    # Equalize the Y channel
    original_yuv[:, :, 0] = cv2.equalizeHist(original_yuv[:, :, 0])
    new_ad_yuv[:, :, 0] = cv2.equalizeHist(new_ad_yuv[:, :, 0])

    # Convert back to BGR
    original_frame_adj = cv2.cvtColor(original_yuv, cv2.COLOR_YUV2BGR)
    new_ad_adj = cv2.cvtColor(new_ad_yuv, cv2.COLOR_YUV2BGR)

    return original_frame_adj, new_ad_adj

def replace_advertisement(frame, bbox, new_ad):
    x1, y1, x2, y2 = bbox
    # Resize the advertisement to fit the bounding box
    new_ad_resized = cv2.resize(new_ad, (int(x2 - x1), int(y2 - y1)))

    # Ensure the new ad is placed within the frame
    frame[y1:y2, x1:x2] = new_ad_resized

    return frame

def process_video_for_ad_replacement(video_path, output_path, ad_image_path, detected_bboxes):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 30.0, (640, 480))

    new_ad = load_advertisement(ad_image_path)

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Adjust lighting if necessary
        original_frame_adj, new_ad_adj = adjust_lighting(frame, new_ad)

        # Assume detected_bboxes contains bbox for the current frame
        # For this example, we'll use a placeholder list
        for bbox in detected_bboxes:  # Replace with actual detection logic
            original_frame_adj = replace_advertisement(original_frame_adj, bbox, new_ad_adj)

        # Write the processed frame to the output video
        out.write(original_frame_adj)

    cap.release()
    out.release()

# Example usage for three videos
ad_image_path = 'image.jpeg'  # Path to your new advertisement image
detected_bboxes_example = [[50, 100, 300, 200], [400, 100, 600, 250]]  # Replace with actual bounding boxes from your detection

for video in ['normalized_video1.mp4', 'normalized_video2.mp4', 'normalized_video3.mp4']:
    process_video_for_ad_replacement(video, f'output_replaced_{video}', ad_image_path, detected_bboxes_example)


In [11]:
# Define output paths explicitly
output_path1 = "path/to/your/output_directory/output_replaced_normalized_video1.mp4"
output_path2 = "path/to/your/output_directory/output_replaced_normalized_video2.mp4"
output_path3 = "path/to/your/output_directory/output_replaced_normalized_video3.mp4"


In [18]:
import os

output_file = "output_normalized_video3.mp4"  # Change to your output file name

# Get file size
file_size = os.path.getsize(output_file)
print(f"File size: {file_size} bytes")


File size: 258 bytes


In [19]:
import cv2

# Define video codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or cv2.VideoWriter_fourcc(*'XVID') for .avi
output_file = 'output_replaced_normalized_video_1.mp4'
out = cv2.VideoWriter(output_file, fourcc, 30.0, (640, 480))  # Adjust dimensions as needed

# Make sure to include your frame processing here
# Example:
for frame in frames:  # frames should be a list of your processed frames
    out.write(frame)  # Write the frame to the video

out.release()  # Don't forget to release the video writer


NameError: name 'frames' is not defined

In [20]:
pip install opencv-python


Note: you may need to restart the kernel to use updated packages.


In [23]:
import cv2

def normalize_video(input_video_path, output_video_path, width, height):
    cap = cv2.VideoCapture(input_video_path)
    
    fourcc = cv2.VideoWriter_fourcc(*'XVID')  # You can also try 'mp4v' or others
    out = cv2.VideoWriter(output_video_path, fourcc, 30.0, (width, height))

    if not cap.isOpened():
        print("Error: Could not open video.")
        return

    while True:
        ret, frame = cap.read()
        if not ret:
            print("End of video or failed to read frame.")
            break
        
        print(f"Processing frame: {cap.get(cv2.CAP_PROP_POS_FRAMES)}")
        frame_resized = cv2.resize(frame, (width, height))
        out.write(frame_resized)

    cap.release()
    out.release()
    print("Video processing complete.")

# Usage
normalize_video('video1.mp4', 'normalized_video.mp4', 640, 480)


Processing frame: 1.0
Processing frame: 2.0
Processing frame: 3.0
Processing frame: 4.0
Processing frame: 5.0
Processing frame: 6.0
Processing frame: 7.0
Processing frame: 8.0
Processing frame: 9.0
Processing frame: 10.0
Processing frame: 11.0
Processing frame: 12.0
Processing frame: 13.0
Processing frame: 14.0
Processing frame: 15.0
Processing frame: 16.0
Processing frame: 17.0
Processing frame: 18.0
Processing frame: 19.0
Processing frame: 20.0
Processing frame: 21.0
Processing frame: 22.0
Processing frame: 23.0
Processing frame: 24.0
Processing frame: 25.0
Processing frame: 26.0
Processing frame: 27.0
Processing frame: 28.0
Processing frame: 29.0
Processing frame: 30.0
Processing frame: 31.0
Processing frame: 32.0
Processing frame: 33.0
Processing frame: 34.0
Processing frame: 35.0
Processing frame: 36.0
Processing frame: 37.0
Processing frame: 38.0
Processing frame: 39.0
Processing frame: 40.0
Processing frame: 41.0
Processing frame: 42.0
Processing frame: 43.0
Processing frame: 44